# Experiment 042: Prediction Calibration

**Problem:** CV-LB relationship is LB = 4.29*CV + 0.0528. The intercept (0.0528) > Target (0.0347), meaning even CV=0 would give LB > target.

**Hypothesis:** The high intercept may be due to systematic bias in predictions. Calibration could reduce this.

**Approaches to test:**
1. Platt scaling (sigmoid calibration)
2. Isotonic regression
3. Temperature scaling
4. Constant offset adjustment
5. Stronger regularization (higher dropout, weight decay)

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.isotonic import IsotonicRegression
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["SM", "Product 2", "Product 3"]]
    return X, Y

print('Data loading functions defined')

In [ ]:
# Load feature lookup tables
spange_df = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
drfp_df = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)

SPANGE_COLS = [c for c in spange_df.columns if c != 'solvent smiles']
DRFP_COLS = [c for c in drfp_df.columns if str(c).isdigit() or isinstance(c, int)]

print(f'Spange: {len(SPANGE_COLS)} features')
print(f'DRFP: {len(DRFP_COLS)} features')

In [ ]:
# Load data
X_single, Y_single = load_data('single_solvent')
X_full, Y_full = load_data('full')

print(f'Single solvent: {len(X_single)} samples')
print(f'Full data: {len(X_full)} samples')

In [ ]:
# Baseline model from exp_030 (GP+MLP+LGBM ensemble)
# This is our best model with CV 0.008298

class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16]):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.Dropout(0.3)
            ])
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, 3))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

print('MLPModel defined')

In [ ]:
# Feature extraction function
def get_features(X, data_type='single'):
    """Extract features for a dataframe."""
    features_list = []
    
    for idx, row in X.iterrows():
        # Kinetics features
        time_m = row['Residence Time']
        temp_c = row['Temperature']
        temp_k = temp_c + 273.15
        
        kinetics = np.array([
            time_m,
            temp_c,
            1.0 / temp_k,
            np.log(time_m + 1),
            time_m / temp_k
        ], dtype=np.float32)
        
        if data_type == 'single':
            solvent = row['SOLVENT NAME']
            spange = spange_df.loc[solvent, SPANGE_COLS].values.astype(np.float32) if solvent in spange_df.index else np.zeros(len(SPANGE_COLS), dtype=np.float32)
            drfp = drfp_df.loc[solvent, DRFP_COLS].values.astype(np.float32) if solvent in drfp_df.index else np.zeros(len(DRFP_COLS), dtype=np.float32)
        else:
            solvent_a = row['SOLVENT A NAME']
            solvent_b = row['SOLVENT B NAME']
            pct_b = row['SolventB%'] / 100.0
            
            sp_a = spange_df.loc[solvent_a, SPANGE_COLS].values.astype(np.float32) if solvent_a in spange_df.index else np.zeros(len(SPANGE_COLS), dtype=np.float32)
            sp_b = spange_df.loc[solvent_b, SPANGE_COLS].values.astype(np.float32) if solvent_b in spange_df.index else np.zeros(len(SPANGE_COLS), dtype=np.float32)
            spange = (1 - pct_b) * sp_a + pct_b * sp_b
            
            dr_a = drfp_df.loc[solvent_a, DRFP_COLS].values.astype(np.float32) if solvent_a in drfp_df.index else np.zeros(len(DRFP_COLS), dtype=np.float32)
            dr_b = drfp_df.loc[solvent_b, DRFP_COLS].values.astype(np.float32) if solvent_b in drfp_df.index else np.zeros(len(DRFP_COLS), dtype=np.float32)
            drfp = (1 - pct_b) * dr_a + pct_b * dr_b
        
        features = np.concatenate([kinetics, spange, drfp])
        features_list.append(features)
    
    return np.array(features_list, dtype=np.float32)

print('Feature extraction defined')

In [ ]:
# Ensemble model with calibration option
class CalibratedEnsembleModel:
    def __init__(self, data='single', gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.3, 
                 calibration='none', dropout=0.3, weight_decay=1e-4):
        self.data_type = data
        self.gp_weight = gp_weight
        self.mlp_weight = mlp_weight
        self.lgbm_weight = lgbm_weight
        self.calibration = calibration
        self.dropout = dropout
        self.weight_decay = weight_decay
        
        self.scaler = None
        self.gp_models = []
        self.mlp_models = []
        self.lgbm_models = []
        self.calibrators = []  # For isotonic regression
    
    def train_model(self, X_train, y_train, epochs=200):
        X_feat = get_features(X_train, self.data_type)
        y_np = y_train.values.astype(np.float32)
        
        # Scale features
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_feat)
        
        # Train GP models (one per target)
        self.gp_models = []
        for i in range(3):
            kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=2, random_state=42)
            gp.fit(X_scaled[:, :18], y_np[:, i])  # Use only Spange + kinetics for GP
            self.gp_models.append(gp)
        
        # Train MLP models (ensemble of 3)
        self.mlp_models = []
        for _ in range(3):
            model = MLPModel(X_scaled.shape[1], hidden_dims=[32, 16]).to(device)
            # Modify dropout if specified
            for module in model.modules():
                if isinstance(module, nn.Dropout):
                    module.p = self.dropout
            
            optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=self.weight_decay)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
            
            X_tensor = torch.tensor(X_scaled).to(device)
            y_tensor = torch.tensor(y_np).to(device)
            
            dataset = TensorDataset(X_tensor, y_tensor)
            loader = DataLoader(dataset, batch_size=32, shuffle=True)
            
            model.train()
            for epoch in range(epochs):
                for X_batch, y_batch in loader:
                    optimizer.zero_grad()
                    pred = model(X_batch)
                    # Weighted loss: [1, 1, 2] for SM
                    weights = torch.tensor([1.0, 1.0, 2.0]).to(device)
                    loss = (weights * (pred - y_batch)**2).mean()
                    loss.backward()
                    optimizer.step()
                scheduler.step()
            
            model.eval()
            self.mlp_models.append(model)
        
        # Train LGBM models (one per target)
        self.lgbm_models = []
        for i in range(3):
            lgbm_model = lgb.LGBMRegressor(
                n_estimators=100,
                learning_rate=0.05,
                max_depth=5,
                num_leaves=31,
                random_state=42,
                verbose=-1
            )
            lgbm_model.fit(X_scaled, y_np[:, i])
            self.lgbm_models.append(lgbm_model)
        
        return self
    
    def predict(self, X_test):
        X_feat = get_features(X_test, self.data_type)
        X_scaled = self.scaler.transform(X_feat)
        
        # GP predictions
        gp_preds = np.zeros((len(X_test), 3))
        for i, gp in enumerate(self.gp_models):
            gp_preds[:, i] = gp.predict(X_scaled[:, :18])
        
        # MLP predictions (average of ensemble)
        mlp_preds = []
        for model in self.mlp_models:
            X_tensor = torch.tensor(X_scaled).to(device)
            with torch.no_grad():
                pred = model(X_tensor).cpu().numpy()
            mlp_preds.append(pred)
        mlp_preds = np.mean(mlp_preds, axis=0)
        
        # LGBM predictions
        lgbm_preds = np.zeros((len(X_test), 3))
        for i, lgbm_model in enumerate(self.lgbm_models):
            lgbm_preds[:, i] = lgbm_model.predict(X_scaled)
        
        # Ensemble
        ensemble_preds = self.gp_weight * gp_preds + self.mlp_weight * mlp_preds + self.lgbm_weight * lgbm_preds
        
        # Clip to valid range
        ensemble_preds = np.clip(ensemble_preds, 0, 1)
        
        return torch.tensor(ensemble_preds, dtype=torch.float32)

print('CalibratedEnsembleModel defined')

In [ ]:
# Test baseline on single fold
test_solvent = sorted(X_single["SOLVENT NAME"].unique())[0]
mask = X_single["SOLVENT NAME"] != test_solvent

print(f"Test solvent: {test_solvent}")
print(f"Training samples: {mask.sum()}, Test samples: {(~mask).sum()}")

# Baseline model
model_baseline = CalibratedEnsembleModel(data='single', gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.3)
model_baseline.train_model(X_single[mask], Y_single[mask], epochs=150)
preds_baseline = model_baseline.predict(X_single[~mask])

actuals = Y_single[~mask].values
mse_baseline = np.mean((actuals - preds_baseline.numpy())**2)
print(f"\nBaseline MSE: {mse_baseline:.6f}")

In [ ]:
# Test with stronger regularization
print("\nTesting stronger regularization...")

# Higher dropout (0.5 instead of 0.3)
model_dropout = CalibratedEnsembleModel(data='single', gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.3, dropout=0.5)
model_dropout.train_model(X_single[mask], Y_single[mask], epochs=150)
preds_dropout = model_dropout.predict(X_single[~mask])
mse_dropout = np.mean((actuals - preds_dropout.numpy())**2)
print(f"Dropout 0.5: MSE = {mse_dropout:.6f}")

# Higher weight decay (1e-3 instead of 1e-4)
model_wd = CalibratedEnsembleModel(data='single', gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.3, weight_decay=1e-3)
model_wd.train_model(X_single[mask], Y_single[mask], epochs=150)
preds_wd = model_wd.predict(X_single[~mask])
mse_wd = np.mean((actuals - preds_wd.numpy())**2)
print(f"Weight decay 1e-3: MSE = {mse_wd:.6f}")

# Both
model_both = CalibratedEnsembleModel(data='single', gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.3, dropout=0.5, weight_decay=1e-3)
model_both.train_model(X_single[mask], Y_single[mask], epochs=150)
preds_both = model_both.predict(X_single[~mask])
mse_both = np.mean((actuals - preds_both.numpy())**2)
print(f"Both (dropout 0.5 + wd 1e-3): MSE = {mse_both:.6f}")

In [ ]:
# Test with different GP weights
print("\nTesting different GP weights...")

for gp_w in [0.0, 0.2, 0.3, 0.4, 0.5]:
    mlp_w = (1 - gp_w) * 0.65  # Maintain MLP/LGBM ratio
    lgbm_w = (1 - gp_w) * 0.35
    
    model = CalibratedEnsembleModel(data='single', gp_weight=gp_w, mlp_weight=mlp_w, lgbm_weight=lgbm_w)
    model.train_model(X_single[mask], Y_single[mask], epochs=150)
    preds = model.predict(X_single[~mask])
    mse = np.mean((actuals - preds.numpy())**2)
    print(f"GP={gp_w:.1f}, MLP={mlp_w:.2f}, LGBM={lgbm_w:.2f}: MSE = {mse:.6f}")